In [155]:
import pandas as pd
from nba_api.stats.endpoints import commonteamroster
from matplotlib import pyplot as plt
import numpy as np 
import datetime
import os
from nba_api.stats.static import players
import time
from typing import List, Iterable, Set, Dict, Optional
from nba_api.stats.endpoints import CommonTeamRoster




### what this can be made into

i can create a system where i put out player projections for a player


# Overall

csv's ->
nba monthly schedule --> will update this with full schedule -> and add month filter later
clusters csv ->
teams clsutered with their team name and cluster
player logs ->
logs up to jan 5th, needed to be dynamic enough where it updates after every day
merges -> todays schedule <-- clusters <-- team ids
utilities --> f
iltering functions, creating new columns, reading files i


## Schedule Blocks

reads schedule file
creates new columns for mascot based on home/visior column


In [156]:
preclean_sched = pd.read_csv('nbaschedjan.csv')

preclean_sched['Visitor Mascot'] = preclean_sched['Visitor/Neutral'].str.split().str[-1]

preclean_sched['Home Mascot'] = preclean_sched['Home/Neutral'].str.split().str[-1]

keep = ['Date','Visitor/Neutral', "Home/Neutral", 'Visitor Mascot', "Home Mascot"]

keep_cols = [c for c in keep if c in preclean_sched.columns]
schedv1 = preclean_sched.loc[:, keep_cols].copy(deep=True)
schedv1

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot
0,"Thu, Jan 1, 2026",Houston Rockets,Brooklyn Nets,Rockets,Nets
1,"Thu, Jan 1, 2026",Miami Heat,Detroit Pistons,Heat,Pistons
2,"Thu, Jan 1, 2026",Philadelphia 76ers,Dallas Mavericks,76ers,Mavericks
3,"Thu, Jan 1, 2026",Boston Celtics,Sacramento Kings,Celtics,Kings
4,"Thu, Jan 1, 2026",Utah Jazz,Los Angeles Clippers,Jazz,Clippers
...,...,...,...,...,...
229,"Sat, Jan 31, 2026",San Antonio Spurs,Charlotte Hornets,Spurs,Hornets
230,"Sat, Jan 31, 2026",Atlanta Hawks,Indiana Pacers,Hawks,Pacers
231,"Sat, Jan 31, 2026",New Orleans Pelicans,Philadelphia 76ers,Pelicans,76ers
232,"Sat, Jan 31, 2026",Minnesota Timberwolves,Memphis Grizzlies,Timberwolves,Grizzlies


In [193]:
today = pd.Timestamp.today().normalize()
print(today)
schedv1["Date"].dtype
schedv1["Date"] = pd.to_datetime(schedv1["Date"])
schedv1["Date"].dtype
todays_schedule = schedv1[schedv1["Date"] == today]
todays_schedule



2026-01-17 00:00:00


,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot
121,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks
122,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks
123,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons
124,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks
125,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat
126,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs
127,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors
128,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets
129,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers


## cluster data

creating mascot column for pre clustered teams file
merges clusters table into todays schedule table on team mascot (home and away)


In [194]:
### making a visitor/home cluster column

clusters_csv = pd.read_csv('clusters - Sheet1.csv')
clusters_csv['TEAM_MASCOT'] = clusters_csv['TEAM_NAME'].str.split().str[-1]
clusters_csv

,TEAM_ID,TEAM_NAME,cluster,TEAM_MASCOT
0,1610612737,Atlanta Hawks,0,Hawks
1,1610612738,Boston Celtics,2,Celtics
2,1610612751,Brooklyn Nets,1,Nets
3,1610612766,Charlotte Hornets,1,Hornets
4,1610612741,Chicago Bulls,3,Bulls
5,1610612739,Cleveland Cavaliers,0,Cavaliers
6,1610612742,Dallas Mavericks,1,Mavericks
7,1610612743,Denver Nuggets,5,Nuggets
8,1610612765,Detroit Pistons,0,Pistons
9,1610612744,Golden State Warriors,1,Warriors


In [195]:
todays_schedule = todays_schedule.copy()

todays_schedule = todays_schedule.merge(
    clusters_csv[["TEAM_MASCOT", "cluster"]],
    left_on="Visitor Mascot",
    right_on="TEAM_MASCOT",
    how="left"
).rename(columns={"cluster": "Visitor Cluster"}).drop(columns=["TEAM_MASCOT"])
todays_schedule = todays_schedule.merge(
    clusters_csv[["TEAM_MASCOT", "cluster"]],
    left_on="Home Mascot",
    right_on="TEAM_MASCOT",
    how="left"
).rename(columns={"cluster": "Home Cluster"}).drop(columns=["TEAM_MASCOT"])




In [196]:
todays_schedule

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1


### Now adding team ids


In [197]:
todays_schedule = todays_schedule.merge(
    clusters_csv[["TEAM_MASCOT", "TEAM_ID"]],
    left_on="Visitor Mascot",
    right_on="TEAM_MASCOT",
    how="left"
).rename(columns={"TEAM_ID": "Visitor TEAM_ID"}).drop(columns=["TEAM_MASCOT"])


In [198]:
todays_schedule

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster,Visitor TEAM_ID
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1,1610612762
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0,1610612738
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0,1610612754
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2,1610612756
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3,1610612760
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1,1610612750
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1,1610612766
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5,1610612764
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1,1610612747


In [199]:
todays_schedule = todays_schedule.merge(
    clusters_csv[["TEAM_MASCOT", "TEAM_ID"]],
    left_on="Home Mascot",
    right_on="TEAM_MASCOT",
    how="left"
).rename(columns={"TEAM_ID": "Home TEAM_ID"}).drop(columns=["TEAM_MASCOT"])
todays_schedule

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster,Visitor TEAM_ID,Home TEAM_ID
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1,1610612762,1610612742
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0,1610612738,1610612737
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0,1610612754,1610612765
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2,1610612756,1610612752
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3,1610612760,1610612748
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1,1610612750,1610612759
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1,1610612766,1610612744
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5,1610612764,1610612743
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1,1610612747,1610612757


## Final todays schedule table

final table represents the current days schedule with each home and aways team id and cluster


In [200]:
todays_schedule

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster,Visitor TEAM_ID,Home TEAM_ID
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1,1610612762,1610612742
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0,1610612738,1610612737
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0,1610612754,1610612765
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2,1610612756,1610612752
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3,1610612760,1610612748
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1,1610612750,1610612759
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1,1610612766,1610612744
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5,1610612764,1610612743
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1,1610612747,1610612757


In [201]:
todays_full_card = todays_schedule.copy(deep=True)
todays_full_card

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster,Visitor TEAM_ID,Home TEAM_ID
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1,1610612762,1610612742
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0,1610612738,1610612737
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0,1610612754,1610612765
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2,1610612756,1610612752
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3,1610612760,1610612748
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1,1610612750,1610612759
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1,1610612766,1610612744
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5,1610612764,1610612743
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1,1610612747,1610612757


## player logs

now we use use player logs from from the nba api for the current season, this was pulled for the current season


In [202]:
player_logs = pd.read_parquet("player_logs_2025_26.parquet")
player_logs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12108 entries, 0 to 12107
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SEASON_YEAR        12108 non-null  object        
 1   GAME_ID            12108 non-null  object        
 2   GAME_DATE          12108 non-null  datetime64[ns]
 3   PLAYER_ID          12108 non-null  int64         
 4   PLAYER_NAME        12108 non-null  object        
 5   TEAM_ID            12108 non-null  int64         
 6   TEAM_ABBREVIATION  12108 non-null  object        
 7   TEAM_NAME          12108 non-null  object        
 8   MATCHUP            12108 non-null  object        
 9   WL                 12108 non-null  object        
 10  MIN                12108 non-null  float64       
 11  MIN_SEC            12108 non-null  object        
 12  PTS                12108 non-null  int64         
 13  REB                12108 non-null  int64         
 14  OREB  

In [203]:
from nba_api.stats.static import teams as nba_teams

teams_list = nba_teams.get_teams()
abbrev_to_id = {t["abbreviation"]: t["id"] for t in teams_list}


adding opp team ids to the player logs table for lookup


In [204]:

def add_opp_team_id(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # opponent abbrev is last token in "BOS vs LAL" or "BOS @ LAL"
    df["OPP_TEAM_ABBREVIATION"] = df["MATCHUP"].str.split().str[-1]

    df["OPP_TEAM_ID"] = df["OPP_TEAM_ABBREVIATION"].map(abbrev_to_id)

    # Optional: sanity check
    missing = df[df["OPP_TEAM_ID"].isna()]["OPP_TEAM_ABBREVIATION"].unique()
    if len(missing) > 0:
        print("⚠️ Unmapped opponent abbreviations:", missing)

    return df

player_logs = add_opp_team_id(player_logs)


### completed player logs table with opp ids


In [205]:
player_logs

,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
0,2025-26,0022500528,2026-01-08,1630163,LaMelo Ball,1610612766,CHA,Charlotte Hornets,CHA vs. IND,L,...,3,0,3,3,3,19,0,0,IND,1610612754
1,2025-26,0022500531,2026-01-08,1642843,Cooper Flagg,1610612742,DAL,Dallas Mavericks,DAL @ UTA,L,...,3,1,3,4,2,3,1,0,UTA,1610612762
2,2025-26,0022500530,2026-01-08,203944,Julius Randle,1610612750,MIN,Minnesota Timberwolves,MIN vs. CLE,W,...,0,0,4,3,4,31,1,0,CLE,1610612739
3,2025-26,0022500528,2026-01-08,1627783,Pascal Siakam,1610612754,IND,Indiana Pacers,IND @ CHA,W,...,0,0,6,2,2,7,1,0,CHA,1610612766
4,2025-26,0022500531,2026-01-08,1628374,Lauri Markkanen,1610612762,UTA,Utah Jazz,UTA vs. DAL,W,...,0,2,3,2,3,7,0,0,DAL,1610612742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12103,2025-26,0022500002,2025-10-21,1629020,Jarred Vanderbilt,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,1,0,2,0,0,-4,0,0,GSW,1610612744
12104,2025-26,0022500002,2025-10-21,1642366,Quinten Post,1610612744,GSW,Golden State Warriors,GSW @ LAL,W,...,0,0,0,1,1,5,0,0,LAL,1610612747
12105,2025-26,0022500001,2025-10-21,1642959,Chris Youngblood,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,0,0,0,0,0,0,0,HOU,1610612745
12106,2025-26,0022500001,2025-10-21,1642964,Brooks Barnhizer,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,0,0,0,0,-2,0,0,HOU,1610612745


### clusters with team ids returned as lists


In [206]:
def load_clusters_csv(path: str, cluster_col: str = "cluster", team_id_col: str = "TEAM_ID") -> pd.DataFrame:
    df = pd.read_csv(path)
    # normalize types
    df[team_id_col] = df[team_id_col].astype(int)
    df[cluster_col] = df[cluster_col].astype(int)
    return df

def build_cluster_to_team_ids_from_df(df: pd.DataFrame, cluster_col: str = "cluster", team_id_col: str = "TEAM_ID"):
    return df.groupby(cluster_col)[team_id_col].apply(list).to_dict()


clusters_df = load_clusters_csv("clusters - Sheet1.csv")
cluster_to_team_ids = build_cluster_to_team_ids_from_df(clusters_df)
cluster_to_team_ids


{0: [1610612737,
  1610612739,
  1610612765,
  1610612745,
  1610612746,
  1610612755,
  1610612761],
 1: [1610612751,
  1610612766,
  1610612742,
  1610612744,
  1610612754,
  1610612750,
  1610612753,
  1610612757,
  1610612759],
 2: [1610612738, 1610612749, 1610612752],
 3: [1610612741, 1610612763, 1610612748, 1610612760, 1610612764],
 4: [1610612740, 1610612762],
 5: [1610612743, 1610612747, 1610612756, 1610612758]}

In [15]:
def get_team_ids_for_cluster(cluster_to_team_ids: dict, k: int):
    return cluster_to_team_ids.get(k, [])
def get_team_roster_player_ids(team_id: int, season: str = "2025-26") -> List[int]:
    df = CommonTeamRoster(team_id=team_id, season=season).get_data_frames()[0]
    if df.empty:
        return []
    return df["PLAYER_ID"].dropna().astype(int).tolist()


### gets team rosters player ids, and caches them


In [16]:
_roster_cache: Dict[tuple, List[int]] = {}

def get_team_roster_player_ids_cached(team_id: int, season: str = "2025-26", sleep_s: float = 0.6, verbose: bool = True) -> List[int]:
    key = (team_id, season)
    if key in _roster_cache:
        return _roster_cache[key]

    try:
        ids = get_team_roster_player_ids(team_id=team_id, season=season)
        _roster_cache[key] = ids
        if verbose:
            print(f"Roster cached: team {team_id} -> {len(ids)} players")
        time.sleep(sleep_s)
        return ids
    except Exception as e:
        if verbose:
            print(f"❌ Roster error team {team_id}: {e}")
        _roster_cache[key] = []
        time.sleep(sleep_s)
        return []


In [17]:
def build_player_pool_from_team_ids(team_ids: Iterable[int], season: str = "2025-26", sleep_s: float = 0.6) -> List[int]:
    mega: Set[int] = set()
    for tid in team_ids:
        mega.update(get_team_roster_player_ids_cached(tid, season=season, sleep_s=sleep_s))
    return sorted(mega)


In [19]:
def filter_player_vs_cluster(
    player_logs: pd.DataFrame,
    player_ids: list,
    opponent_team_ids: list,
    min_minutes: float = 0.0,
) -> pd.DataFrame:
    df = player_logs[
        (player_logs["PLAYER_ID"].isin(player_ids)) &
        (player_logs["OPP_TEAM_ID"].isin(opponent_team_ids))
    ]
    if min_minutes > 0:
        df = df[df["MIN"] >= min_minutes]
    return df.sort_values("GAME_DATE")


In [207]:
todays_full_card

,Date,Visitor/Neutral,Home/Neutral,Visitor Mascot,Home Mascot,Visitor Cluster,Home Cluster,Visitor TEAM_ID,Home TEAM_ID
0,2026-01-17,Utah Jazz,Dallas Mavericks,Jazz,Mavericks,4,1,1610612762,1610612742
1,2026-01-17,Boston Celtics,Atlanta Hawks,Celtics,Hawks,2,0,1610612738,1610612737
2,2026-01-17,Indiana Pacers,Detroit Pistons,Pacers,Pistons,1,0,1610612754,1610612765
3,2026-01-17,Phoenix Suns,New York Knicks,Suns,Knicks,5,2,1610612756,1610612752
4,2026-01-17,Oklahoma City Thunder,Miami Heat,Thunder,Heat,3,3,1610612760,1610612748
5,2026-01-17,Minnesota Timberwolves,San Antonio Spurs,Timberwolves,Spurs,1,1,1610612750,1610612759
6,2026-01-17,Charlotte Hornets,Golden State Warriors,Hornets,Warriors,1,1,1610612766,1610612744
7,2026-01-17,Washington Wizards,Denver Nuggets,Wizards,Nuggets,3,5,1610612764,1610612743
8,2026-01-17,Los Angeles Lakers,Portland Trail Blazers,Lakers,Blazers,5,1,1610612747,1610612757


### infractstructure to read todays schedule and automatically generate team v cluster df


### needs to read a row, create a dictionary and have the key as the cluster (for home and away)

### so read visitor cluster, key is cluster, add visitor team id to value, whihc should be a list

### then we do this for both columns and loop thru the keys in the dictionary, use that as a param to index the clusters sheet and

### use the team ids to filter all teams vs that cluster, they then should create dataframes that are titles teamsvscluster1,2,3,4,5, etc ``


In [150]:
from collections import defaultdict

def build_cluster_to_teams_today(schedule_df):
    """
    Returns:
      dict[int, list[int]]
      cluster -> teams playing against that cluster today
    """
    cluster_map = defaultdict(set)

    for _, row in schedule_df.iterrows():
        # visitor team faces home cluster
        cluster_map[row["Home Cluster"]].add(row["Visitor TEAM_ID"])

        # home team faces visitor cluster
        cluster_map[row["Visitor Cluster"]].add(row["Home TEAM_ID"])

    # convert sets to sorted lists
    return {k: sorted(v) for k, v in cluster_map.items()}


In [208]:
cluster_to_team_ids_today = build_cluster_to_teams_today(todays_full_card)

for cluster, team_ids in cluster_to_team_ids_today.items():
    print(f"Cluster {cluster} <- teams:", team_ids)


Cluster 1 <- teams: [1610612744, 1610612747, 1610612750, 1610612759, 1610612762, 1610612765, 1610612766]
Cluster 4 <- teams: [1610612742]
Cluster 0 <- teams: [1610612738, 1610612754]
Cluster 2 <- teams: [1610612737, 1610612756]
Cluster 5 <- teams: [1610612752, 1610612757, 1610612764]
Cluster 3 <- teams: [1610612743, 1610612748, 1610612760]


In [213]:
cluster_to_team_ids_today

{1: [1610612744,
  1610612747,
  1610612750,
  1610612759,
  1610612762,
  1610612765,
  1610612766],
 4: [1610612742],
 0: [1610612738, 1610612754],
 2: [1610612737, 1610612756],
 5: [1610612752, 1610612757, 1610612764],
 3: [1610612743, 1610612748, 1610612760]}

In [209]:
cluster_player_dfs = {}

for opp_cluster, team_ids in cluster_to_team_ids_today.items():
    df = filter_player_vs_cluster(
        player_logs,
        player_ids=build_player_pool_from_team_ids(team_ids),
        opponent_team_ids=cluster_to_team_ids[opp_cluster],
        min_minutes=15
    )

    cluster_player_dfs[opp_cluster] = df


In [210]:
df_c0 = cluster_player_dfs[0]

df_c1 = cluster_player_dfs[1]

# df_c2 = cluster_player_dfs[2]

df_c3 = cluster_player_dfs[3]

df_c4 = cluster_player_dfs[4]

df_c5 = cluster_player_dfs[5]


# display(df_c3)


In [211]:
df_c1

,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
12100,2025-26,0022500002,2025-10-21,1629216,Gabe Vincent,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,1,0,0,3,1,-5,0,0,GSW,1610612744
12068,2025-26,0022500002,2025-10-21,1629029,Luka Dončić,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,2,1,3,1,10,-8,1,0,GSW,1610612744
12071,2025-26,0022500002,2025-10-21,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,2,0,5,5,10,-14,0,0,GSW,1610612744
12099,2025-26,0022500002,2025-10-21,1631222,Jake LaRavia,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,0,0,0,1,1,-5,0,0,GSW,1610612744
12090,2025-26,0022500002,2025-10-21,1629028,Deandre Ayton,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,0,1,4,3,1,-4,0,0,GSW,1610612744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2025-26,0022500531,2026-01-08,203937,Kyle Anderson,1610612762,UTA,Utah Jazz,UTA vs. DAL,W,...,2,0,0,1,1,-4,0,0,DAL,1610612742
43,2025-26,0022500528,2026-01-08,1642883,Sion James,1610612766,CHA,Charlotte Hornets,CHA vs. IND,L,...,0,0,1,2,2,3,0,0,IND,1610612754
53,2025-26,0022500528,2026-01-08,1630182,Josh Green,1610612766,CHA,Charlotte Hornets,CHA vs. IND,L,...,1,0,1,1,0,8,0,0,IND,1610612754
28,2025-26,0022500528,2026-01-08,1629012,Collin Sexton,1610612766,CHA,Charlotte Hornets,CHA vs. IND,L,...,2,0,3,4,6,-22,0,0,IND,1610612754


In [212]:


df_c5


,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
11099,2025-26,0022500122,2025-10-27,1631104,Blake Wesley,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,0,0,1,3,3,-5,0,0,LAL,1610612747
10923,2025-26,0022500122,2025-10-27,1630166,Deni Avdija,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,1,3,3,2,2,-2,0,0,LAL,1610612747
10925,2025-26,0022500122,2025-10-27,201950,Jrue Holiday,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,1,2,2,3,1,19,0,0,LAL,1610612747
10940,2025-26,0022500122,2025-10-27,203924,Jerami Grant,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,3,1,2,1,3,1,0,0,LAL,1610612747
11071,2025-26,0022500122,2025-10-27,1631200,Kris Murray,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,2,0,0,2,1,7,0,0,LAL,1610612747
10958,2025-26,0022500122,2025-10-27,1631101,Shaedon Sharpe,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,4,0,5,2,2,21,0,0,LAL,1610612747
11059,2025-26,0022500122,2025-10-27,1641739,Toumani Camara,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,0,1,2,2,4,4,0,0,LAL,1610612747
10942,2025-26,0022500122,2025-10-27,1642270,Donovan Clingan,1610612757,POR,Portland Trail Blazers,POR @ LAL,W,...,1,2,2,2,2,21,1,0,LAL,1610612747
10350,2025-26,0022500026,2025-10-31,1630166,Deni Avdija,1610612757,POR,Portland Trail Blazers,POR vs. DEN,W,...,0,0,4,5,5,17,0,0,DEN,1610612743
10361,2025-26,0022500026,2025-10-31,1641739,Toumani Camara,1610612757,POR,Portland Trail Blazers,POR vs. DEN,W,...,2,2,1,5,3,-2,0,0,DEN,1610612743


### old lookup below


In [218]:
my_team_ids = [1610612737]  # paste your list for player lookup 1630559-> austin reaves
mega_player_ids = build_player_pool_from_team_ids(my_team_ids, season="2025-26")

# {1: [1610612744,
#   1610612747,
#   1610612750,
#   1610612759,
#   1610612762,
#   1610612765,
#   1610612766],
#  4: [1610612742],
#  0: [1610612738, 1610612754],
#  2: [1610612737, 1610612756],
#  5: [1610612752, 1610612757, 1610612764],
#  3: [1610612743, 1610612748, 1610612760]

target_cluster = 5
opp_ids = cluster_to_team_ids[target_cluster]

df1 = filter_player_vs_cluster(
    player_logs,
    player_ids=[1630559],
    opponent_team_ids=[1610612744,
  1610612747,
  1610612750,
  1610612759,
  1610612762,
  1610612765,
  1610612766],
    min_minutes=15
)

df2 = filter_player_vs_cluster(
    player_logs,
    player_ids=[1630559],
    opponent_team_ids=[1610612737, 1610612756],
    min_minutes=15
)

df3 = filter_player_vs_cluster(
    player_logs,
    player_ids=[1630559],
    opponent_team_ids=[1610612743, 1610612748, 1610612760],
    min_minutes=15
)

df4 = filter_player_vs_cluster(
    player_logs,
    player_ids=[1630559],
    opponent_team_ids=[1610612742],
    min_minutes=15
)


# display(df.head())

# display(
#     df[["PLAYER_NAME"]]
#         .drop_duplicates()
#         .sort_values(["PLAYER_NAME"])
# )



In [219]:
df1

,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
12071,2025-26,0022500002,2025-10-21,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,...,2,0,5,5,10,-14,0,0,GSW,1610612744
11498,2025-26,0022500019,2025-10-24,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL vs. MIN,W,...,0,0,1,1,5,17,1,0,MIN,1610612750
10581,2025-26,0022500136,2025-10-29,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL @ MIN,W,...,2,0,3,1,8,-1,1,0,MIN,1610612750
8643,2025-26,0022500198,2025-11-10,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL @ CHA,W,...,2,0,1,2,8,18,0,0,CHA,1610612766
7391,2025-26,0022500253,2025-11-18,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL vs. UTA,W,...,1,0,4,3,8,10,0,0,UTA,1610612762
6596,2025-26,0022500282,2025-11-23,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL @ UTA,W,...,1,0,3,2,4,-1,1,0,UTA,1610612762
4184,2025-26,0022501204,2025-12-10,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,LAL vs. SAS,L,...,0,1,3,5,3,-5,0,0,SAS,1610612759


In [227]:
# df1[['MIN_SEC', 'PTS']].head()
df1 = df1[df1['MIN'] > 0]  # protect against divide-by-zero

df1['PTS_PER_MIN'] = df1['PTS'] / df1['MIN']
df1[['MIN','PTS', 'PTS_PER_MIN']]


,MIN,PTS,PTS_PER_MIN
12071,36.333333,26,0.715596
11498,36.583333,25,0.683371
10581,39.645000,28,0.706268
8643,29.416667,24,0.815864
7391,33.433333,26,0.777667
6596,39.966667,22,0.550459
4184,39.816667,15,0.376727


In [228]:
rate_summary = {
    'games': len(df1),
    'mean_ppm': df1['PTS_PER_MIN'].mean(),
    'median_ppm': df1['PTS_PER_MIN'].median(),
    'std_ppm': df1['PTS_PER_MIN'].std(),
    'q25_ppm': df1['PTS_PER_MIN'].quantile(0.25),
    'q75_ppm': df1['PTS_PER_MIN'].quantile(0.75),
}

pd.Series(rate_summary)



games         7.000000
mean_ppm      0.660850
median_ppm    0.706268
std_ppm       0.150640
q25_ppm       0.616915
q75_ppm       0.746632
dtype: float64

In [233]:
expected_minutes = 32

expected_points = {
    'mean_pts': rate_summary['mean_ppm'] * expected_minutes,
    'median_pts': rate_summary['median_ppm'] * expected_minutes,
    'low_band_pts': rate_summary['q25_ppm'] * expected_minutes,
    'high_band_pts': rate_summary['q75_ppm'] * expected_minutes,
}

pd.Series(expected_points)


mean_pts         21.147210
median_pts       22.600580
low_band_pts     19.741280
high_band_pts    23.892213
dtype: float64

In [137]:
my_team_ids = [1610612747,1610612741
]  # paste your list for player lookup
mega_player_ids = build_player_pool_from_team_ids(my_team_ids, season="2025-26")

target_cluster = 0
opp_ids = cluster_to_team_ids[target_cluster]

df_2 = filter_player_vs_cluster(
    player_logs,
    player_ids=mega_player_ids,
    opponent_team_ids=opp_ids,
    min_minutes=15
)

# display(df.head())

display(
    df_2[["PLAYER_NAME"]]
        .drop_duplicates()
        .sort_values(["PLAYER_NAME"])
)



,PLAYER_NAME
6287,Austin Reaves
11907,Ayo Dosunmu
9067,Bronny James
3609,Coby White
9048,Dalton Knecht
9069,Deandre Ayton
6358,Gabe Vincent
12000,Isaac Okoro
9028,Jake LaRavia
9717,Jalen Smith


In [138]:
df_2



,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
12000,2025-26,0022500084,2025-10-22,1630171,Isaac Okoro,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,...,0,0,2,5,3,17,0,0,DET,1610612765
11806,2025-26,0022500084,2025-10-22,202696,Nikola Vučević,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,...,1,0,1,2,2,8,1,0,DET,1610612765
11830,2025-26,0022500084,2025-10-22,1630581,Josh Giddey,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,...,0,1,6,5,7,6,1,0,DET,1610612765
11843,2025-26,0022500084,2025-10-22,1641824,Matas Buzelis,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,...,0,3,3,3,3,-10,0,0,DET,1610612765
11962,2025-26,0022500084,2025-10-22,1630172,Patrick Williams,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,...,1,2,2,1,2,1,0,0,DET,1610612765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2025-26,0022500517,2026-01-07,1628989,Kevin Huerter,1610612741,CHI,Chicago Bulls,CHI @ DET,L,...,1,0,3,1,1,-9,0,0,DET,1610612765
150,2025-26,0022500517,2026-01-07,1630200,Tre Jones,1610612741,CHI,Chicago Bulls,CHI @ DET,L,...,0,0,3,2,3,-2,0,0,DET,1610612765
105,2025-26,0022500517,2026-01-07,1630245,Ayo Dosunmu,1610612741,CHI,Chicago Bulls,CHI @ DET,L,...,0,1,1,2,2,-20,0,0,DET,1610612765
92,2025-26,0022500517,2026-01-07,1641824,Matas Buzelis,1610612741,CHI,Chicago Bulls,CHI @ DET,L,...,1,4,2,1,3,-9,0,0,DET,1610612765


In [39]:
my_team_ids = [1610612759]  # paste your list for player lookup
mega_player_ids = build_player_pool_from_team_ids(my_team_ids, season="2025-26")

target_cluster = 2
opp_ids = cluster_to_team_ids[target_cluster]

df_3 = filter_player_vs_cluster(
    player_logs,
    player_ids=mega_player_ids,
    opponent_team_ids=opp_ids,
    min_minutes=15
)

# display(df.head())

display(
    df_3[["PLAYER_NAME"]]
        .drop_duplicates()
        .sort_values(["PLAYER_NAME"])
)



,PLAYER_NAME
